# 라이브러리 import

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K 

from keras.datasets import cifar10

## 데이터 불러오기

In [3]:
NUM_CLASS = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 이미지 데이터 정규화
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, NUM_CLASS)
y_test = to_categorical(y_test, NUM_CLASS)

# CNN 시작

In [6]:
input_layer = Input(shape=(64,64,1))

conv_layer_1 = Conv2D(
    filters = 2,
    kernel_size = (3,3),
    strides = 1,
    padding = 'same')(input_layer)

입력 shape가 (64,64,1)의 값을 가지고, conv층이 위와 같이 구성되어 있으면, 우선 필터개수가 2개이고 커널 사이즈가 (3,3)의 크기에 stride가 1 이고 패딩 사이즈가 same으로 설정되어 있으므로 출력 shape는 (배치 크기, 높이, 너비, 필터 개수) 이므로 (1, 64, 64, 2)의 크기로 반환이 될 것이다.
  
- 스트라이드 : 필터가 이동하는 크기
- 패딩 : same 옵션을 주고 stride를 1로 설정하면, 입력과 출력의 크기를 같게 만들어 줌

In [7]:
# cifar10 데이터에 적용하기
input_layer = Input(shape=(32,32,3))

conv_layer_1 = Conv2D(
    filters = 10,
    kernel_size = (4,4),
    strides = 2,
    padding = 'same')(input_layer)

conv_layer_2 = Conv2D(
    filters = 20,
    kernel_size = (3,3),
    strides = 2,
    padding = 'same')(conv_layer_1)

flatten_layer = Flatten()(conv_layer_2)

output_layer = Dense(units = 10, activation='softmax')(flatten_layer)

model = Model(input_layer, output_layer)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 10)        490       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 20)          1820      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                12810     
Total params: 15,120
Trainable params: 15,120
Non-trainable params: 0
_________________________________________________________________


- padding = 'same' 옵션인 합성곱 층의 출력 크기 : (None, 입력높이/스트라이드, 입력너비/스트라이드, 필터개수)

## 합성곱, 배치정규화, 드롭아웃 적용한 모델 만들기

In [9]:
input_layer = Input(shape=(32,32,3))

x = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Flatten()(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.5)(x)

x = Dense(NUM_CLASS)(x)
output_layer = Activation('softmax')(x)
model = Model(input_layer, output_layer)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 32)        0         
__________

In [11]:
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [12]:
model.fit(x_train
          , y_train
          , batch_size=32
          , epochs=10
          , shuffle=True
          , validation_data = (x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 632s 13ms/step - loss: 1.5692 - acc: 0.4498 - val_loss: 1.2337 - val_acc: 0.5667
Epoch 2/10
50000/50000 [==============================] - 641s 13ms/step - loss: 1.1750 - acc: 0.5844 - val_loss: 1.1258 - val_acc: 0.6007
Epoch 3/10
50000/50000 [==============================] - 634s 13ms/step - loss: 1.0129 - acc: 0.6462 - val_loss: 0.9474 - val_acc: 0.6715
Epoch 4/10
50000/50000 [==============================] - 592s 12ms/step - loss: 0.9243 - acc: 0.6783 - val_loss: 1.3200 - val_acc: 0.5658
Epoch 5/10
50000/50000 [==============================] - 625s 13ms/step - loss: 0.8626 - acc: 0.6978 - val_loss: 0.9578 - val_acc: 0.6652
Epoch 6/10
50000/50000 [==============================] - 577s 12ms/step - loss: 0.8098 - acc: 0.7174 - val_loss: 0.8387 - val_acc: 0.7045
Epoch 7/10
50000/50000 [==============================] - 544s 11ms/

In [13]:
model.evaluate(x_test, y_test, batch_size=1000)

10000/10000 [==============================] - 26s 3ms/step


[0.9108119547367096, 0.6916000127792359]